### launching of SPs



In [5]:
from datetime import timedelta
from decimal import Decimal
from tqdm import tqdm
import pandas as pd

import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc
from sql.sql_requests import SQL_Communicator

In [6]:
def format_time_delta(duration: timedelta):
    # Get the total seconds
    total_seconds = duration.total_seconds()
    # days = duration.days
    hours = duration.seconds // 3600  # Convert seconds to hours
    minutes = (duration.seconds // 60) % 60  # Convert remaining seconds to minutes
    seconds = duration.seconds % 60
    if hours > 0:
        return f'{hours:02d}:{minutes:02d}:{int(seconds):02d}'
    if minutes > 0:
        return f'{minutes:02d}:{int(seconds):02d}'
    return f'{total_seconds:4.2f}'

In [9]:
entity_names =[

"FutureEventMkt_FutureEventMktToObjectToEventType"
]

durations = []
with SQL_Communicator() as sc:
    for en in tqdm(entity_names):
        print(f'flow for target entity {en}..')
        en2 = nc.default_rename(en)
        sp_names = [nc.pull_sp_name(en2), nc.merge_sp_name(en2), nc.merge_direct_sp_name(en2)]
        stage_tbl = nc.stg_table_name(en)
        trg_tbl = nc.table_name(nc.default_rename(en))
        ent_durs, flow_td = [en2], timedelta()  
        for i, sp_name in enumerate(sp_names):
            exec_stm = f"EXEC {sp_name}"
            print(f'- {exec_stm}...')
            try:
                td = sc.get_execution_metrics(exec_stm)
                ent_durs.append(format_time_delta(td))
                if i < 2:
                    flow_td +=td
                if  i == 1:
                    ent_durs.append(format_time_delta(flow_td))
                print(f'  - duration {format_time_delta(td)}')
                if i == 0:  ## pull sp
                    rows_count, table_size = sc.get_table_size(stage_tbl)
                    print(f'rows {rows_count}, stage table size: {table_size}')
                # sc.get_execution_metrics(f'TRUNCATE table {trg_tbl}')
            except Exception as ex:
                print(f'shit happened {ex}')
        rows_count = rows_count /1000
        velocity = table_size / Decimal(flow_td.total_seconds()) if flow_td.total_seconds() else 0 
        ent_durs.extend([f'{rows_count:7.2f}' , f'{table_size:7.2f}', f'{velocity:7.2f}'])
        durations.append(ent_durs)
        # ent_durs.append(sum(ent_durs))


connecting...
Connection successful!


  0%|          | 0/1 [00:00<?, ?it/s]

flow for target entity FutureEventMkt_FutureEventMktToObjectToEventType..
- EXEC dbo.PullData_FutureEventMkt_FutureEventMktToObjectToEventType2_prc...
  - duration 39.30
rows 11379757, stage table size: 419.203125
- EXEC dbo.MergeData_FutureEventMkt_FutureEventMktToObjectToEventType2_prc...
  - duration 48.25
- EXEC dbo.MergeDataDirect_FutureEventMkt_FutureEventMktToObjectToEventType2_prc...


100%|██████████| 1/1 [02:01<00:00, 121.66s/it]

  - duration 33.99


In [10]:
columns = ['Entity name', 'Pull SP', 'Merge SP', 'sum', 'Merge Direct',  'rows_count, K', 'table_size, MB', 'velocity, MB/sec']
df = pd.DataFrame(durations, columns = columns)
# df['table_size, MB'] = df ['table_size, MB'].astype('int')
# df['rows_count, K'] = df['rows_count, K']/1000
df

,Entity name,Pull SP,Merge SP,sum,Merge Direct,"rows_count, K","table_size, MB","velocity, MB/sec"
0,FutureEventMkt_FutureEventMktToObjectToEventType2,39.30,48.25,01:27,33.99,11379.76,419.20,4.79
